<a href="https://colab.research.google.com/github/jiya-0905/Anomaly-Detection-/blob/main/Anomaly_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pycaret &> /dev/null
!pip install sklearn &> /dev/null
!pip install pandas &> /dev/null

In [2]:
import pycaret
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [3]:
# loading sample dataset from pycaret dataset module
from pycaret.datasets import get_data
data = get_data('anomaly')

,Col1,Col2,Col3,Col4,Col5,Col6,Col7,Col8,Col9,Col10
0,0.263995,0.764929,0.138424,0.935242,0.605867,0.518790,0.912225,0.608234,0.723782,0.733591
1,0.546092,0.653975,0.065575,0.227772,0.845269,0.837066,0.272379,0.331679,0.429297,0.367422
2,0.336714,0.538842,0.192801,0.553563,0.074515,0.332993,0.365792,0.861309,0.899017,0.088600
3,0.092108,0.995017,0.014465,0.176371,0.241530,0.514724,0.562208,0.158963,0.073715,0.208463
4,0.325261,0.805968,0.957033,0.331665,0.307923,0.355315,0.501899,0.558449,0.885169,0.182754


In [4]:
# import pycaret anomaly and init setup
from pycaret.anomaly import *
s = setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 10)"
2,Transformed data shape,"(1000, 10)"
3,Numeric features,10
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [5]:
from pycaret.anomaly import AnomalyExperiment
exp = AnomalyExperiment()

In [6]:
# init setup on exp
exp.setup(data, session_id = 123)

,Description,Value
0,Session id,123
1,Original data shape,"(1000, 10)"
2,Transformed data shape,"(1000, 10)"
3,Numeric features,10
4,Preprocess,True
5,Imputation type,simple
6,Numeric imputation,mean
7,Categorical imputation,mode
8,CPU Jobs,-1
9,Use GPU,False


In [7]:
available_models=models()

In [8]:
model_ids = list(available_models.index)

In [9]:
import pandas as pd
result=[]
for m in model_ids[:-2]:
    model = exp.create_model(m)
    anomalies = exp.assign_model(model)

    # Ensure data is in the correct format before prediction
    if not isinstance(data.columns, pd.RangeIndex):
        data = data.reset_index(drop=True)

    pred = exp.predict_model(model, data=data)
    anomaly_count = pred[pred['Anomaly'] == 1].shape[0]
    filtered_data = pred[pred['Anomaly'] == 0]
    X = filtered_data.drop(columns=['Anomaly', 'Anomaly_Score'], errors='ignore')
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    silhouette_scor=0
    n=0
    for i in range(2,11):
        kmeans = KMeans(n_clusters=i, random_state=123)
        cluster_labels = kmeans.fit_predict(X_scaled)
        silhouette = silhouette_score(X_scaled, cluster_labels)
        if silhouette>silhouette_scor:
            silhouette_scor=silhouette
            n=i
    result.append({"Model": m, "Anomaly_Count": anomaly_count, "Clusters": n, "Silhouette_Score": silhouette_scor})

df = pd.DataFrame(result)
df.to_csv("model_metrics.csv", index=False)



Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]

Processing:   0%|          | 0/3 [00:00<?, ?it/s]